In [121]:
from langchain_core.exceptions import OutputParserException


class Model:
    def __init__(self, model_type):
        self.model = Ollama(model=model_type)
        self.response_schemas = [
            ResponseSchema(name="explanation", description="explain which part of the old comment should be modified."),
            ResponseSchema(name="newComment", description="the new comment"),
        ]
        self.output_parser = StructuredOutputParser.from_response_schemas(self.response_schemas)
        self.format_instructions = self.output_parser.get_format_instructions()
        self.prompt = PromptTemplate(
            template=
            '''
            Read the following Java method: ```{old_method}```
            Read the following javadoc comment belonging to the method mentioned before: {old_comment}
            Then the Java method was modified to ```{new_method}```
            Please tell me how should the old comment be changed to fit the new method.
            Output in json format.
            {format_instructions}''',
            input_variables=["old_method", "old_comment", "new_method"],
            partial_variables={"format_instructions": self.format_instructions},
        )
        self.chain = self.prompt | self.model | self.output_parser

    def resolve(self, old_method, new_method, old_comment):
        while True:
            try:
                return self.chain.invoke({
                    'old_method': old_method,
                    'new_method': new_method,
                    'old_comment': old_comment
                })
            except OutputParserException as e:
                print(e)
    

In [123]:
import json

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.llms.ollama import Ollama

with open('../data/test.json', 'r') as raw_json:
    read = raw_json.read()
    parsed = json.loads(read)

myModel = Model('mistral-openorca')
for data in [parsed]:
    _old_method = parsed['src_method']
    _new_method = parsed['dst_method']
    _old_comment = parsed['src_javadoc']
    result = myModel.resolve(_old_method, _new_method, _old_comment)
    print(result)

{'explanation': 'Configure a sslConfig.Builder for the server using the legacy configuration', 'newComment': 'Set up an SSLConfig.Builder with the specified settings according to the legacy distribution config.'}
{'explanation': 'Update the javadoc comment to reflect the change from SSLConfig to SSLConfig.Builder', 'newComment': 'Configure a sslConfig.Builder for the server using the legacy configuration'}
{'explanation': 'The old comment should be changed to fit the new method, which now uses a builder instead of directly setting values.', 'newComment': 'Configure a sslConfig.Builder for the server using the legacy configuration'}
{'explanation': 'The old comment explained that the method was used to configure an sslConfig for the server using the legacy configuration. The method has been updated to use a builder instead.', 'newComment': 'Configure a sslConfig for the server using the legacy configuration with a builder.'}
{'explanation': 'Change the javadoc comment to fit the new met